In [21]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense   ,   LSTM   ,Embedding , Bidirectional

In [2]:
#importer le texte à traiter
#file=open("The Adventures of Roderick Random.txt","r",encoding="utf8")
import requests
texte=requests.get("https://www.gutenberg.org/files/4085/4085-0.txt")

In [3]:
texte.text

'ï»¿The Project Gutenberg eBook of The Adventures of Roderick Random, by Tobias Smollett\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country where you are located before\r\nusing this eBook.\r\n\r\nTitle: The Adventures of Roderick Random\r\n\r\nAuthor: Tobias Smollett\r\n\r\nRelease Date: May, 2003 [eBook #4085]\r\n[Most recently updated: August 5, 2021]\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\nProduced by: Tapio Riikonen and David Widger\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF RODERICK RANDOM ***\r\n\r\n\r\n\r\n\r\nThe Adventures of Roderick Random\r\n\r\nby Tobias Smolle

In [13]:
data=texte.text.split('\n')
data=data[299:]

In [14]:
data=" ".join(data)

In [15]:
#remove special caracters
data=data.replace('\r','').replace('UTF-8','').replace("\'","").replace('"',"").replace(string.punctuation,"")

In [16]:
token=Tokenizer()
token.fit_on_texts([data]) 
seq=token.texts_to_sequences([data])[0]

In [17]:
vocab_size=len(token.word_index)+1

In [18]:
sequences=[]
for i in range(4,len(seq)):
    words=seq[i-4:i+1]
    sequences.append(words)

sequences

[[5, 12, 2014, 8, 1],
 [12, 2014, 8, 1, 4336],
 [2014, 8, 1, 4336, 145],
 [8, 1, 4336, 145, 2],
 [1, 4336, 145, 2, 19],
 [4336, 145, 2, 19, 1403],
 [145, 2, 19, 1403, 3139],
 [2, 19, 1403, 3139, 8],
 [19, 1403, 3139, 8, 1],
 [1403, 3139, 8, 1, 101],
 [3139, 8, 1, 101, 2],
 [8, 1, 101, 2, 7],
 [1, 101, 2, 7, 1039],
 [101, 2, 7, 1039, 6],
 [2, 7, 1039, 6, 137],
 [7, 1039, 6, 137, 2],
 [1039, 6, 137, 2, 748],
 [6, 137, 2, 748, 111],
 [137, 2, 748, 111, 4],
 [2, 748, 111, 4, 1148],
 [748, 111, 4, 1148, 26],
 [111, 4, 1148, 26, 15],
 [4, 1148, 26, 15, 29],
 [1148, 26, 15, 29, 121],
 [26, 15, 29, 121, 1318],
 [15, 29, 121, 1318, 5355],
 [29, 121, 1318, 5355, 86],
 [121, 1318, 5355, 86, 8],
 [1318, 5355, 86, 8, 635],
 [5355, 86, 8, 635, 2],
 [86, 8, 635, 2, 11],
 [8, 635, 2, 11, 194],
 [635, 2, 11, 194, 4],
 [2, 11, 194, 4, 12],
 [11, 194, 4, 12, 1149],
 [194, 4, 12, 1149, 16],
 [4, 12, 1149, 16, 11],
 [12, 1149, 16, 11, 5356],
 [1149, 16, 11, 5356, 8],
 [16, 11, 5356, 8, 1],
 [11, 5356, 8, 1

In [19]:
x=[]
y=[]

for i in sequences:
    x.append(i[:4])
    y.append(i[4])


x=np.array(x)
y=np.array(y)


In [20]:
y=to_categorical(y,num_classes=vocab_size)
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [31]:
model=Sequential()
model.add(Embedding(vocab_size,output_dim=100,input_length=4))
model.add(Bidirectional(LSTM(150, return_sequences = True)))

model.add(LSTM(128))
model.add(Dense(vocab_size/2,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 4, 100)            1294400   
                                                                 
 bidirectional_2 (Bidirectio  (None, 4, 300)           301200    
 nal)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 128)               219648    
                                                                 
 dense_4 (Dense)             (None, 6472)              834888    
                                                                 
 dense_5 (Dense)             (None, 12944)             83786512  
                                                                 
Total params: 86,436,648
Trainable params: 86,436,648
Non-trainable params: 0
__________________________________________

In [33]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [35]:
model.fit(x,y,epochs=18,batch_size=256)

Epoch 1/18
757/757 [==============================] - 627s 828ms/step - loss: 3.3225 - accuracy: 0.2856
Epoch 2/18
757/757 [==============================] - 627s 829ms/step - loss: 2.8329 - accuracy: 0.3525
Epoch 3/18
757/757 [==============================] - 643s 849ms/step - loss: 2.4225 - accuracy: 0.4202
Epoch 4/18
757/757 [==============================] - 761s 1s/step - loss: 2.0791 - accuracy: 0.4834
Epoch 5/18
757/757 [==============================] - 762s 1s/step - loss: 1.7935 - accuracy: 0.5409
Epoch 6/18
757/757 [==============================] - 759s 1s/step - loss: 1.5576 - accuracy: 0.5915
Epoch 7/18
757/757 [==============================] - 765s 1s/step - loss: 1.3588 - accuracy: 0.6383
Epoch 8/18
757/757 [==============================] - 704s 929ms/step - loss: 1.1907 - accuracy: 0.6772
Epoch 9/18
757/757 [==============================] - 626s 827ms/step - loss: 1.0470 - accuracy: 0.7108
Epoch 10/18
757/757 [==============================] - 627s 829ms/step - los

In [95]:
def predict_words(sentence):
  
    
    for i in range(100):
        token_list = token.texts_to_sequences([sentence])[0]
        token_list = pad_sequences([token_list], maxlen=4, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=1)

        new_word = ''

        for word, index in token.word_index.items():
            if predicted == index:
                new_word = word
                break
        sentence += " " + new_word
        
    return sentence   

In [65]:
import tensorflow as tf
import gradio as gr

In [89]:
sentence=gr.inputs.Textbox()


In [92]:
interface=gr.Interface(fn=predict_words, inputs=[sentence , no_words ,maxlen], outputs=output)

In [108]:
interface=gr.Interface(fn=predict_words,inputs=gr.inputs.Textbox(lines=10,placeholder='Write a sentence'),outputs='text')

In [ ]:
interface.launch()